# Blocking with Word2Vec

In [1]:
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## Caricamento dello Schema Mediato da File

In [2]:
schema_mediato = pd.read_csv("./../../datasets/MediatedSchemaSemicolon.csv",sep=";")
print("Dimensioni dello Schema Mediato:", schema_mediato.shape)

Dimensioni dello Schema Mediato: (81706, 6)


In [3]:
records = []
for i in range(0,len(schema_mediato)):
    record = schema_mediato.iloc[i].dropna()
    records.append(record.tolist())

Osservazione: ogni Record è rappresentato con una Lista di Stringhe, dove le 'stringhe' sono i campi che lo caratterizzano.

## Inizializzazione del Modello "Word2Ved"

In [4]:
num_threads = os.cpu_count()
print("Numero di thread disponibili:", num_threads)

Numero di thread disponibili: 8


In [5]:
model = Word2Vec(sentences=records, vector_size=512, window=5, min_count=1, workers=num_threads)

## Calcolo dei Vettori per i Record
Abbiamo settato la dimensione dello Spazio Latente a 256 e per ogni vettore dobbiamo, quindi, ottenere un vettore di 256 valori.

Per un Record il vettore viene ottenuto sommando tra loro "element-wise" i vettori ottenuti dai campi ed infine normalizzando il vettore risultante sul valore 256, pari alla dimensione dello Spazio Latente.

In [6]:
def calculate_vector(record):
    # Inizializza un vettore vuoto
    record_vector = np.zeros(model.vector_size)
    # Somma i vettori di embedding di ciascun token nel record
    for token in record:
        record_vector += model.wv[token]
    #Normalizza il vettore risultante
    record_vector /= 512
    return record_vector

In [7]:
records_w2v = []
for i in range(0, len(records)):
    records_w2v.append(calculate_vector(records[i]))

In [8]:
df = pd.DataFrame(records_w2v)
print("Dimensioni del DataFrame di Vettori:", df.shape)

Dimensioni del DataFrame di Vettori: (81706, 512)


## K-Means sui Vettori per costruire i Blocchi

In [9]:
kmeans = KMeans(n_clusters=836, random_state=0).fit(df)

/Users/alessandropesare/anaconda3/envs/FinalProjectIDD3_8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [10]:
schema_mediato['block'] = kmeans.labels_

In [11]:
schema_mediato.to_csv("./../../datasets/MediatedSchemaConBlocks_W2V.csv", index=False)